In [1]:
begin
    include("../src/utilitarios.jl")
    include("../src/RCRM.jl")
    include("../src/RDRM.jl")
    include("../src/RK.jl")
    include("../src/RandBlock_CRM.jl")  
    include("../src/RrDRM.jl")  
    include("../src/Blockwise.jl")
    using Downloads, Tar, MatrixMarket, SparseArrays,  PGFPlotsX, Plots, DataFrames, CSV, SuiteSparseMatrixCollection
    # Para gráficos melhores, descomente a próxima linha
    # pgfplotsx()
end

In [6]:
begin # Para baixar as matrizes use os seguintes comandos
    matrix_group = "Meszaros"     # Define o grupo da matriz
    matrix_name = "model1"        # Define o nome da matriz
    ssmc = ssmc_db()              # Inicializa o banco de dados local da SuiteSparse
    ssmc_matrices(ssmc, matrix_group, matrix_name) 
    paths = fetch_ssmc(ssmc_matrices(ssmc, matrix_group, matrix_name), format="MM")  # Baixa a matriz no formato Matrix Market
    downloaded_files = installed_ssmc()
    path = paths[1]
    matrix_name = readdir(path)[1]
    T = Float64
    M = float.(mmread(path * "/" * matrix_name))
    A =  M[findall(row -> any(x -> x != 0, row), eachrow(M)), :] #Tira as linhas de zeros
    num_rows, num_cols = size(A)
    if num_rows < num_cols
        A = transpose(A) ### Se a matriz tem mais colunas, tomamos A^T para garantir o sistema sobredeterminado
        num_rows, num_cols = size(A)
    end
    w = rand(num_rows)  ## Cria vetor com distribuição gaussiana.
    x̄ = A'*w / norm(A'*w) ## Cria uma solução que depende do vetor aleatório.
    b = A*x̄ ## Define o lado direito do sistema linear.
    peso = peso_Vershynin(Matrix(A)) # Calcula a probabilidade da norma da linha em relação a norma da matrix ||a_i||^2/||A||_F^2
    A
end

┌ Info: loaded database with revision date
└   last_rev_date = "08-Oct-2020 17:09:58"


798×362 Transpose{Float64, SparseMatrixCSC{Float64, Int64}} with 3028 stored entries:
⎡⠻⣿⣟⠇⡿⣀⠿⠀⣿⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎤
⎢⠀⡿⠤⠄⠅⠏⢾⡿⡿⢧⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⣿⣷⠀⠀⠀⠀⠀⠀⠸⡄⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠻⢻⣤⠆⠀⠀⠀⠀⠀⡇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠻⣇⣀⡀⣀⠀⠀⢇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠄⡿⣼⡇⠐⠀⠀⢸⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⡀⠀⠀⡇⠛⢧⣤⡆⠀⠀⣇⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠃⠀⠙⠘⠃⠀⠀⠸⣶⣶⣰⢰⡆⢰⡆⢰⣆⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⡯⠥⢸⢸⢸⣶⣾⡿⡄⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⣿⡄⠀⠀⠀⠉⠈⠀⢧⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠧⢧⣠⠀⠀⠀⠀⠀⢸⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠸⣷⠀⣀⢀⠀⠀⢸⡀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠠⢼⣏⢿⠈⠅⠀⠀⡇⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⢸⠸⢿⣤⣠⠀⠀⢹⠀⎥
⎢⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠰⣌⠀⠀⠘⠀⠘⠋⠓⠀⠀⠀⠃⎥
⎢⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠱⣄⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⣄⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠓⠄⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⣀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⠀⠀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⠦⡀⠀⠀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠘⢦⡀⠀⠀⠀⠀⎥
⎢⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠳⡄⠀⠀⎥
⎣⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠈⠢⡀⎦

KeyError: KeyError: key "usage_request" not found

In [3]:
function Testes_Matrix(matrix_group, matrix_name, vec_met, vec_r, tol)
    df = DataFrame(Metodo = String[], time = Float64[], iter =Float64[], tol = Float64[],
                   numproj = Float64[], count_error = Float64[])
    ssmc = ssmc_db()
    ssmc_matrices(ssmc, matrix_group, matrix_name) 
    paths =  fetch_ssmc(ssmc_matrices(ssmc, matrix_group, matrix_name), format="MM")
    downloaded_files = installed_ssmc()
    path = paths[1]
    matrix_name = readdir(path)[1]
    T = Float64
    M = float.(mmread(path * "/" * matrix_name))
    A =  M[findall(row -> any(x -> x != 0, row), eachrow(M)), :] 
    num_rows, num_cols = size(A)
    if num_rows < num_cols
        A = transpose(A) ### Se a matriz tem mais colunas, tomamos A^T para garantir o sistema sobredeterminado
        num_rows, num_cols = size(A)
    end
    x0 = ones(num_cols) ## Cria chute inicial, nesse caso, vetor de uns.
    peso = peso_Vershynin(A) # Calcula a probabilidade da norma da linha em relação a norma da matrix ||a_i||^2/||A||_F^2

    for r in vec_r
        for met in vec_met
        	Tempo = Float64[]
            Iter = Float64[]
            Tol = Float64[]
            CE = Float64[]
            NP = Float64[]
            for i = 1:20
                Random.seed!(i)
                w = rand(num_rows)  ## Cria vetor com distribuição gaussiana.
                x̄ = A'*w / norm(A'*w) ## Cria uma solução que depende do vetor aleatório.
                b = A*x̄ ## Define o lado direito do sistema linear.
                
        		T = @belapsed $met($A,$x0,$b,$r,$x̄; itmax = 100000, ϵ = tol , pesos = $peso, true_error = true)
                # Mede o tempo de execução.
        		_, it, tol, numproj, count_error = met(A,x0,b,r,x̄; itmax = 100000, ϵ = tol, pesos = peso, true_error = true)
                #Salva as informações dos testes
                push!(Tempo, T)
                push!(Iter, it)
                push!(Tol, tol)
                push!(CE, count_error)
                push!(NP, numproj)
            end
            println("$(met)_$(r) terminou") # Aviso que o método terminou.
    		push!(df, ["$(met)_$(r)", mean(Tempo), mean(Iter), mean(Tol), mean(NP), mean(CE)])
            CSV.write("./SSMC/test_$(matrixname)_1e-12_$(met)_$(r).csv", df) # Salvo o DataFrames
        end
	end
	return println("Acabou")
end

Testes_Matrix (generic function with 1 method)

In [35]:
begin
    matrix_group = "Meszaros"  ## Exemplo
    matrix_name = "model1"     ## Exemplo
    vec_met = [Randblock_CRM, BlockwiseCRM, Rand_CRM_rsets, Rand_DRM_rsets, Rand_mDRM_rsets, Rand_Kaczmarz] # Vetor com os métodos
    vec_r = [2,3,5,10,20,50] # Vetor com os valores de r
    tol = 1e-6 # Define a tolerância desejada.
    Testes_Matrix(matrix_group, matrix_name, vec_met, vec_r, tol) 
    # inicia o teste com a matriz, os métodos e o valor de r selecionados
end

┌ Info: loaded database with revision date
└   last_rev_date = "08-Oct-2020 17:09:58"


LoadError: InterruptException: